### Research Collection Analysis

In [7]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
# from langchain.agents import create_agent  # its deprecated now
from langchain.agents import AgentExecutor, create_tool_calling_agent
 

# initiate the LLM - OpenSource Model(from Groq)
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0) # model="llama-3.3-70b-versatile"
# llm.invoke("Hi")

In [16]:
### Tool 1 - Tavily Search from langchain

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results = 5,
    topic = "general"
)

#agent_tavily = create_agent(llm, [tavily_search_tool])





In [ ]:
"""
### Tool 2 - GoogleAPISerper from langchain

os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool

search = GoogleSerperAPIWrapper()

google_serper_search_tool = Tool(
    name="search",
    func=search.run,
    description="useful for when you need to ask with search"
)

# agent_google = create_agent(llm, [google_serper_search_tool])
"""


In [ ]:
### Tool 2 - Semantic Search from langchain
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.utilities.semanticscholar import SemanticScholarAPIWrapper
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub

instructions = """You are an expert researcher."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize Semantic Scholar Search
tools_semantic_search = [SemanticScholarQueryRun()]
agent = create_tool_calling_agent(llm, tools_semantic_search, prompt)

agent_semantic = AgentExecutor(
    agent = agent,
    tools = tools_semantic_search,
    verbose=True
)

"""
agent_semantic.invoke(
    {
        "input": "What are some biases in the large language models? How have people tried to mitigate them? "
        "show me a list of papers and techniques. Based on your findings write new research questions "
        "to work on. Break down the task into subtasks for search. Use the search tool"
    }
)
"""

'\nagent_executor.invoke(\n    {\n        "input": "What are some biases in the large language models? How have people tried to mitigate them? "\n        "show me a list of papers and techniques. Based on your findings write new research questions "\n        "to work on. Break down the task into subtasks for search. Use the search tool"\n    }\n)\n'

In [ ]:
### Tool 3 - Youtube url search

from langchain_community.tools import YouTubeSearchTool
youtube_tool = YouTubeSearchTool()

In [ ]:
# all_tools = [tavily_search_tool, tools_semantic_search, youtube_tool]

In [ ]:
def collect_and_explain_resources(topic):
    collected_data = ""


    # semantic scholar search
    semantic_query = f"query: basis on topic {topic}, create 3 selections: paper name with URL, summary and citations. "
    print("searching for semantic results")
    semantic_agent_response = agent_semantic.invoke({"input": semantic_query}) # invoke the agent function here
    print("got semantic results")
    collected_data += f"### Semantic Scholar Results: \n{semantic_agent_response}\n\n "


    # YouTube Search
    print("searching for youtube results")
    youtube_results = youtube_tool.run(f"{topic}, 3") # get top 3 youtube results
    print("got youtube results")
    collected_data += f"### YouTube Results :\n {youtube_results} \n\n"


    # Tavily Search
    print("searching for Tavily Search")
    web_search_query = f"find 3 articles/blogs on {topic} and get back top 3 articles/blogs with their names and URL for each. "
    web_results = tavily_search_tool.invoke({"query": web_search_query})
    print("got the Tavily Search result")
    collected_data += f"### Web Articles and Blogs: \n{web_results} \n\n"


    prompt_search = ChatPromptTemplate.from_messages(
        [
            ("system",
            """ You are professional research expert and you have deep experience in any research topic. The User will give you a topic to research. 
            Your job is to analyze the topic using tools and give summary as instructed below, 
            1. You will use Semantic Scholar Search to get paper name with URL, summary and citation 
            2. You will use YouTube Tool to get top 3 url's which are relevant to topic 
            3. You will use Tavily Search to get the details from internet and return top 3 blogs/articals with their names and URL for each

            Based on above, you will summarize the information sequetially by followiing pointers 1, 2 and 3.  

            Here is the data to organize {collected_data}\n
            Please ensure that final output is well structured and easy to understand
          
            """
            ),
            ("user","Get the final structured output")
            ])

    # Format the prompt with collected data
    formatted_output = prompt_search.format_prompt(collected_data=collected_data)
    # generate the response
    structured_output = llm(formatted_output.to_messages())


    #structured_output = llm.generate([prompt_search]).generations[0][0].text

    result = structured_output.content



In [ ]:

topic = "impact on polar bear due to changes in enviornment"
collected_data = ""


# semantic scholar search
semantic_query = f"query: basis on topic {topic}, create 3 selections: paper name with URL, summary and citations. "
print("searching for semantic results")
semantic_agent_response = agent_semantic.invoke({"input": semantic_query}) # invoke the agent function here
print("got semantic results")
collected_data += f"### Semantic Scholar Results: \n{semantic_agent_response}\n\n "


# YouTube Search
print("searching for youtube results")
youtube_results = youtube_tool.run(f"{topic}, 3") # get top 3 youtube results
print("got youtube results")
collected_data += f"### YouTube Results :\n {youtube_results} \n\n"


# Tavily Search
print("searching for Tavily Search")
web_search_query = f"find 3 articles/blogs on {topic} and get back top 3 articles/blogs with their names and URL for each. "
web_results = tavily_search_tool.invoke({"query": web_search_query})
print("got the Tavily Search result")
collected_data += f"### Web Articles and Blogs: \n{web_results} \n\n"


prompt_search = ChatPromptTemplate.from_messages(
    [
        ("system",
        """ You are professional research expert and you have deep experience in any research topic. The User will give you a topic to research. 
        Your job is to analyze the topic using tools and give summary as instructed below, 
        1. You will use Semantic Scholar Search to get paper name with URL, summary and citation 
        2. You will use YouTube Tool to get top 3 url's which are relevant to topic 
        3. You will use Tavily Search to get the details from internet and return top 3 blogs/articals with their names and URL for each

        Based on above, you will summarize the information sequetially by followiing pointers 1, 2 and 3.  

        Here is the data to organize {collected_data}\n
        Please ensure that final output is well structured and easy to understand
          
        """
        ),
        ("user","Get the final structured output")
        ])

# Format the prompt with collected data
formatted_output = prompt_search.format_prompt(collected_data=collected_data)
# generate the response
structured_output = llm(formatted_output.to_messages())


#structured_output = llm.generate([prompt_search]).generations[0][0].text

result = structured_output.content


print("printing result")
print(result)
print("printing structured_output")
print(structured_output)




searching for semantic results


> Entering new AgentExecutor chain...

Invoking: `semanticscholar` with `{'query': 'polar bear environmental impact climate change'}`


Published year: None
Title: How does climate change impact the population of polar bears? Environmental threats, ecological dynamics and conservation efforts
Authors: Dongjin Kim, Jenni Barrera
Abstract: None


Published year: 2012
Title: Modelling of Physical Processes and Assessment of Climate Change Impacts in Great Bear Lake
Authors: Y. Rao, A. Huang, W. Schertzer, W. Rouse
Abstract: None


Published year: 2015
Title: Can polar bears use terrestrial foods to offset lost ice‐based hunting opportunities?
Authors: K. Rode, C. Robbins, Lynne Nelson, S. Amstrup
Abstract: None


Published year: 2014
Title: Role of an Anthropomorphic Agent in Environmental Decision Making
Authors: Molly M. Liechty, Poornima Madhavan
Abstract: None


Published year: 2024
Title: The Impacts of Arctic Climate Change on Polar Bear Biodiversity

C:\Users\Dell\AppData\Local\Temp\ipykernel_16540\3666393122.py:50: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  structured_output = llm(formatted_output.to_messages())


printing result
**Research Report: Impact of Environmental Changes on Polar Bears**

**Semantic Scholar Results:**

Based on the search results, here are three selections:

1. **Paper Name:** "The Impacts of Arctic Climate Change on Polar Bear Biodiversity and Associated Analysis"
**URL:** Not provided
**Summary:** This paper reviews the impacts of climate change on polar bear biodiversity, examining changes to their distribution, habitat use, food supply, and population numbers. It highlights the urgent need to study environmental conservation issues further and proposes initiatives to preserve the polar bear population.
**Citations:** Jiayu Wang (2024)

2. **Paper Name:** "Polar Bear Behavior in Response to Climate Change"
**URL:** Not provided
**Summary:** This paper discusses how polar bears are responding to climate change, including changes in their feeding behavior, movements, and habitat associations. It highlights the importance of behavioral plasticity in polar bears and the 

In [ ]:
"""
from youtube_search import YoutubeSearch

results = YoutubeSearch("LangChain tutorial", max_results=3).to_dict()
print(results)
"""

[{'id': 'nAmC7SoVLd8', 'thumbnails': ['https://i.ytimg.com/vi/nAmC7SoVLd8/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLBQNaKJyG7N0Q504_6akOZ-uSdQxw', 'https://i.ytimg.com/vi/nAmC7SoVLd8/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCegDn4BzbdkQNfOL_51XAHzBABpg'], 'title': 'LangChain Crash Course For Beginners | LangChain Tutorial', 'long_desc': None, 'channel': 'codebasics', 'duration': '46:07', 'views': '516,060 views', 'publish_time': '2 years ago', 'url_suffix': '/watch?v=nAmC7SoVLd8&pp=ygUSTGFuZ0NoYWluIHR1dG9yaWFs'}, {'id': 'cQUUkZnyoD0', 'thumbnails': ['https://i.ytimg.com/vi/cQUUkZnyoD0/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLARpcmF2wUJC2E-Vznc26w7Ffrz1A', 'https://i.ytimg.com/vi/cQUUkZnyoD0/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCaz4v67Hts49T_jOUqIFUvqY0_1Q'], 'title': 'LangChain Tutorial for Beginners | Generative AI Series 🔥', 'long_desc': None, 'channel': 'Cod